## Preambule

In [27]:
# General packages
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import xarray as xr
import json

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Parameter

In [28]:
hist_startyear = 1990

## Paths

In [29]:
path_main = Path("X:/user/dekkerm/Projects/ECEMF_T5.2/")
path_data = path_main / "Data" / str(hist_startyear)
path_figs = path_main / "Figures"

## Read data files

In [30]:
xr_total = xr.open_dataset(path_data / "xr_total.nc")
all_regions_iso = np.load(path_data / "all_regions.npy")
all_regions_names = np.load(path_data / "all_regions_names.npy")
all_countries_iso = np.load(path_data / "all_countries.npy", allow_pickle=True)
all_countries_names = np.load(path_data / "all_countries_names.npy", allow_pickle=True)
only_regions_iso = ['CVF', 'G20', "EU", "G7", "NA", "AU", "AF", "SIDS", "LDC", "WORLD"]
all_categories = ['C1+C2', 'C3', 'C6', 'C7']
only_regions_names = ['Climate Vulnerability Forum','G20',"European Union","G7","Northern America","Australasia","African Group","Small Island Developing States","Least Developed Countries"]

In [31]:
xr_budgets_static = xr.open_dataset(path_data / "xr_budgets_static.nc")
#xr_budgets_linear = xr.open_dataset(path_data / "xr_budgets_linear.nc")
xr_budgets_scenario = xr.open_dataset(path_data / "xr_budgets_scenario.nc")
df_michel = pd.read_excel(Path("X:/user/dekkerm/Data/") / "NDC" / "NDC_comparison.xlsx", sheet_name="Michel")

In [32]:
df_g = pd.read_excel(Path("X:/user/dekkerm/Data/") / "UNFCCC_Parties_Groups_noeu.xlsx", sheet_name = "Country groups")
countries_iso = np.array(df_g["Country ISO Code"])
countries_name = np.array(df_g["Name"])
countries_iso = np.array(df_g["Country ISO Code"])
group_cvf = countries_iso[np.array(df_g["CVF (24/10/22)"]) == 1]
group_g20 = countries_iso[np.array(df_g["G20"]) == 1]
group_eu = countries_iso[np.array(df_g["EU"]) == 1]
group_g7 = countries_iso[np.array(df_g["G7"]) == 1]
group_na = countries_iso[np.array(df_g["Northern America"]) == 1]
group_um = countries_iso[np.array(df_g["Umbrella"]) == 1]
group_au = countries_iso[np.array(df_g["Australasia"]) == 1]
group_af = countries_iso[np.array(df_g["African Group"]) == 1]
group_sids = countries_iso[np.array(df_g["SIDS"]) == 1]
group_ldc = countries_iso[np.array(df_g["LDC"]) == 1]
group_eg = countries_iso[np.array(df_g["European Group"]) == 1]
group_world = np.copy(countries_iso)

## Preprocess

In [33]:
R = [["GF", "Grandfathering", "navy"],
     ["PC", "Per Capita (immediate)", "forestgreen"],
     ["PCC", "Per Capita Convergence", "red"],
     ["F1g", "Approach 1 - GDP (net)", "orange"],
     ["F1h", "Approach 1 - GDP (HDI)", "purple"],
     ["F2", "Approach 2 - GDP (net)", "silver"],
     ["F2C", "Approach 2 - transition", "goldenrod"],
     ]
R = np.array(R).T
colors = R[2]
rules = R[0]
rulenames = R[1]

In [34]:
ar_gig = np.zeros(shape=(len(all_categories), len(rules)+5, len(all_regions_iso)))+np.nan

for c in tqdm(range(len(all_categories))):
    ccat = all_categories[c]
    for cty_i, cty in enumerate(all_regions_iso):
        ar_gig[c, 0, cty_i] = float(xr_total.sel(Time=2019, ISO=cty).GHG_p)

        # # Ratios
        # if total_kyoto_2020 > 0:
        #     ratio = total_co2_2020/total_kyoto_2020
        #     if cty == "CHN":
        #         ratio = 1
        #     ar_gig[c, 1, cty_i] = ratio
        
        # Curpol
        df = df_michel[df_michel.ISO == cty]['curpol_2030']
        if len(df) > 0:
            em_curpol = float(df_michel[df_michel.ISO == cty]['curpol_2030'])/1e3
            ar_gig[c, 2, cty_i] = em_curpol

        # NDC
        ndc_malte = [float(xr_total.sel(ISO=cty, Time=2030).NDC_l), float(xr_total.sel(ISO=cty, Time=2030).NDC_h)]
        ar_gig[c, 3, cty_i] = np.nanmean(ndc_malte)
        
        # Effort sharing rules
        for es_i, es in enumerate(rules):
            try:
                ar_gig[c, es_i+4, cty_i] = float(xr_budgets_scenario.sel(ISO=cty, Time=2030, Category=ccat)[es])
            except:
                ar_gig[c, es_i+4,cty_i] = np.nan
        
        # Scenarios
        # spendsw = []
        # for c in range(8):
        #     spent = float(xr_total.sel(Category="C"+str(c+1), Time=2030).GHG_f)
        #     spendsw.append(spent)
        # spendsw = np.array(spendsw)
        # spends = np.zeros(8)
        # for k in range(8):
        #     try:
        #         spends[k] = float(xr_ar6_iso3.sel(Region=cty, Variable='Emissions|Kyoto Gases|w/o LULUCF', Category="C"+str(k+1), Time=2030).Mean/1e3)
        #     except:
        #         spends[k] = np.nan
        # spends_new = fill_scenemissions(spends, spendsw)
        ar_gig[c, len(rules)+4, cty_i] = np.nan#float(xr_total.sel(Time=2030, ISO=cty, Category=ccat).GHG_f)
xr_preprocess = xr.DataArray(data=ar_gig*1e3,
                            dims=["Ccat", "Variable", "Region"],
                            coords=dict(Ccat=all_categories,
                                        Variable=["Emissions", "Ratio", "CurPol", "NDC"]+list(rules)+["Scenario"],
                                        Region=all_regions_iso))

  0%|          | 0/4 [00:00<?, ?it/s]C:\Users\dekkerm\AppData\Local\Temp\2\ipykernel_4128\3919548340.py:23: RuntimeWarning:

Mean of empty slice

100%|██████████| 4/4 [00:12<00:00,  3.14s/it]


In [35]:
cs = np.zeros(shape=(len(rules), len(all_countries_iso)))
for r_i, r in enumerate(all_countries_iso):
    ndc = float((xr_total.NDC_l.sel(Time=2030, ISO=r) + xr_total.NDC_h.sel(Time=2030, ISO=r))/2)
    for es_i, es in enumerate(rules):
        if not np.isnan(ndc):
            wh = np.where(ndc <= xr_budgets_scenario.sel(Time=2030, ISO=r, Category=all_categories)[es])[0]
            if len(wh) > 0:
                cs[es_i, r_i] = int(wh[0])
            else:
                cs[es_i, r_i] = int(3)
        else:
            cs[es_i, r_i] = np.nan
dicty = {}
dicty["Region"] = all_countries_iso
dicty["C_GF"] = cs[0]
dicty["C_PC"] = cs[1]
dicty["C_PCC"] = cs[2]
dicty["C_F1g"] = cs[3]
dicty["C_F1h"] = cs[4]
dicty["C_F2"] = cs[5]
dicty["C_F2C"] = cs[6]
df_ndc = pd.DataFrame(dicty)

eu_assessment = np.zeros(len(rules))+np.nan
eu_ndc = float((xr_total.NDC_l.sel(Time=2030, ISO='EU') + xr_total.NDC_h.sel(Time=2030, ISO='EU'))/2)
for es_i, es in enumerate(rules):
    wh = np.where(eu_ndc <= xr_budgets_scenario.sel(Time=2030, ISO='EU', Category=all_categories)[es])[0]
    if len(wh) > 0:
        eu_assessment[es_i] = int(wh[0])
    else:
        eu_assessment[es_i] = int(3)

# cs = np.zeros(shape=(len(rules), len(all_countries_iso)))
# for r_i, r in enumerate(all_countries_iso):
#     curpol = float(xr_preprocess.sel(Variable="CurPol", Region=r, Ccat="C1+C2"))
#     for es_i, es in enumerate(rules):
#         ess = np.array(xr_preprocess.sel(Variable=es, Region=r)).astype(float)
#         if not np.isnan(curpol):
#             error = ess-curpol
#             wh = np.where(error >= 0)[0]
#             if len(wh) > 0:
#                 cs[es_i, r_i] = int(wh[0])
#             else:
#                 cs[es_i, r_i] = int(7)
#         else:
#             cs[es_i, r_i] = np.nan
# dicty = {}
# dicty["Region"] = all_countries_iso
# dicty["C_GF"] = cs[0]
# dicty["C_PC"] = cs[1]
# dicty["C_PCC"] = cs[2]
# dicty["C_F1g"] = cs[3]
# dicty["C_F1h"] = cs[4]
# dicty["C_F2"] = cs[5]
# dicty["C_F2C"] = cs[6]
# df_curpol = pd.DataFrame(dicty)

## Plot

In [36]:
gf = np.array([df_ndc.iloc[c]['C_GF'] for c in range(len(df_ndc))])
gf[np.isnan(gf)] = 3
gf = gf.astype(int)

a1g = np.array([df_ndc.iloc[c]['C_F1g'] for c in range(len(df_ndc))])
a1g[np.isnan(a1g)] = 3
a1g = a1g.astype(int)

a1h = np.array([df_ndc.iloc[c]['C_F1h'] for c in range(len(df_ndc))])
a1h[np.isnan(a1h)] = 3
a1h = a1h.astype(int)

a2g = np.array([df_ndc.iloc[c]['C_F2'] for c in range(len(df_ndc))])
a2g[np.isnan(a2g)] = 3
a2g = a2g.astype(int)

pc = np.array([df_ndc.iloc[c]['C_PC'] for c in range(len(df_ndc))])
pc[np.isnan(pc)] = 3
pc = pc.astype(int)

a2t = np.array([df_ndc.iloc[c]['C_F2C'] for c in range(len(df_ndc))])
a2t[np.isnan(a2t)] = 3
a2t = a2t.astype(int)

In [37]:
fig = make_subplots(rows=1, cols=1, subplot_titles=(''), 
                                                    specs=[[{'type': 'choropleth'}]],
                                                    horizontal_spacing = 0, vertical_spacing=0.03)

colorbar_abs = dict(len=1, x=1, tickvals = [0, 1, 2, 3], ticktext = ['1', '10', '100', '1000'])
ts = ['Below 1.5 degrees', 'Well below 2 degrees', 'Below 3 degrees', 'Below 4 degrees']
colorbar_pc = dict(len=1, x=1)

cm = True
t = pc
#t[t >= 4] = np.nan
fig.add_trace(
    go.Choropleth(
    locations=all_countries_iso,
    z = t,
    locationmode = 'ISO-3',
    colorscale = 'RdYlGn_r',
    zmax = 3,
    zmin = 0,
    text = [cty+": "+ts[t[cty_i]] for cty_i, cty in enumerate(all_countries_iso)],
    hovertemplate  = '%{text}',
    name="",
    marker_line_color='white', 
    marker_line_width=0.4,
    colorbar=dict(len=0.6, x=0., y=0.40, tickvals = [0, 1, 2, 3], ticktext = [all_categories[c]+"<br>("+ts[c]+")" for c in range(len(all_categories))], tickfont={'size': 15}),
    showscale=cm
), 1, 1)

euval = eu_assessment[rules == "PC"][0]
fig.add_trace(
    go.Choropleth(
    locations=group_eu,
    z = np.array([euval]*len(group_eu)),
    locationmode = 'ISO-3',
    colorscale = 'RdYlGn_r',
    zmax = 3,
    zmin = 0,
    text = ["EU : "+ts[int(euval)]]*len(group_eu),
    hovertemplate  = '%{text}',
    name="",
    marker_line_color='white', 
    marker_line_width=0.4,
    showscale=False
), 1, 1)

    # euval = eu_assessment[rules == ['F1g', 'F2'][var_i]][0]
    # fig.add_trace(
    #     go.Choropleth(
    #     locations=np.array(["SO"]),
    #     z = np.array([euval]),
    #     locationmode = 'ISO-3',
    #     colorscale = 'RdYlGn_r',
    #     zmax = 3,
    #     zmin = 0,
    #     text = ["Somaliland : "+ts[int(euval)]],
    #     hovertemplate  = '%{text}',
    #     name="",
    #     marker_line_color='white', 
    #     marker_line_width=0.4,
    #     showscale=False
    # ), [1, 2][var_i], [1, 1][var_i])

fig.update_geos(showocean=True, oceancolor="aliceblue")
fig.update_geos(showlakes=True, lakecolor="aliceblue")

fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18),
                   y=1.05, x=0.5, text="<b>NDC alignment of an equal per capita approach</b>")
fig.add_annotation(xref='paper', yref='paper',ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=18),
                   y=0.77, x=0, text="<b>Climate target")

# limite map scope to USA 'africa', 'asia', 'europe', 'north america', 'south america', 'usa', 'world'
fig.update_geos(visible=False,
                showlakes=True,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                showcoastlines=False,)
#fig.update_traces(showscale=False)
fig.update_layout(height=800, width=1500)

In [38]:
fig.write_image("X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Figure_NDCperf/Figure_NDC_yann_PC.svg")
fig.write_image("X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Figure_NDCperf/Figure_NDC_yann_PC.png", scale=3)

import os
from datetime import date
try:
    os.remove('X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Figure_NDCperf/Figure_NDC_yann_PC.html')
except:
    3
def html_w(typ):
    return '<html> '+typ+' <p style="font-family: Arial">'

today = date.today()
td = today.strftime("%B %d, %Y")

with open('X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Figure_NDCperf/Figure_NDC_yann_PC.html', 'a') as f:
    #f.write(html_w('<h1>')+'Figure 1 - Electricity Generation</p></h1>')
    f.write(html_w('<body>')+'Version: '+td+'</p></body>')
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))